In [8]:
import numpy as np
from sympy import symbols, Eq, solve
from sympy import real_root
import json
from scipy.optimize import fsolve
import nbimporter
from CreateCAM import set_camera_location,set_camera_orientation
from PicLocating import RGB_location
   

# Define initial variables

with open('CamID.json', 'r') as file:
    cameras = json.load(file)
for camera in cameras:
    if (camera.get("name")) == 'EricsPad':
        pV=camera.get('pixelVector')
        
#these should later be inputs
a = np.array([4.4, 0.8, 20.5])/20.5
b = np.array([4.4, 1.6, 20.5])/20.5
c = np.array([5.2, 1.6, 20.5])/20.5
a_pixel=a/pV
b_pixel=b/pV
c_pixel=c/pV

AB_square = 4
AC_square = 8
BC_square = 4

tt1=0.0
tt2=0.0
tt3=0.0

In [10]:
def t_to_tCheck(t1,camName,a_pixel,b_pixel,c_pixel,AB_square = 4,AC_square = 8,BC_square = 4,filename='CamID.json'):
    
    with open(filename, 'r') as file:
        cameras = json.load(file)
    for camera in cameras:
        if (camera.get("name")) == camName:
            pixelVector=np.array(camera.get('pixelVector'))
            break
#    print(type(a_pixel))    
#    print(type(pixelVector))        
    a=a_pixel*pixelVector
    b=b_pixel*pixelVector
    c=c_pixel*pixelVector
    
    t1s=np.array([], dtype=float)
    t2s=np.array([], dtype=float)
    t3s=np.array([], dtype=float)
    
    
    # Define symbolic variables for t2, t3, and t1_check
    t1_check, t2, t3 = symbols('t1_check t2 t3', real=True)

    A = a * t1
    eqAB = Eq(np.sum((b * t2 - A) ** 2), AB_square)
    # Solve for t2 based on eqAB
    T2_values = solve(eqAB, t2)
#    print(T2_values)
    for T2 in T2_values:
        if T2.is_real:
            t2s=np.append(t2s,T2)
            B = b * float(T2)
            eqBC = Eq(np.sum((c * t3 - B) ** 2), BC_square)

            # Solve for t3 based on eqBC
            T3_values = solve(eqBC, t3)
 #           print(T3_values)
            for T3 in T3_values:
                
                if T3.is_real:
                    t3s=np.append(t3s,T2)
                    C = c * float(T3)
                    eqAC = Eq(np.sum((C - a * t1_check) ** 2), AC_square)

                    # Solve for t1_check based on eqAC
                    T1_values = solve(eqAC, t1_check)
                    for T1 in T1_values:
                        t1s=np.append(t1s,T1)
    index=np.argmin(abs(t1s-t1))    
    error=float(t1s[index])
    error=(error-t1)/t1   
    print("Error=",100*error,"%")     
    global tt1,tt2,tt3
    tt1= t1s[index]
    tt2= t2s[index//4]
    tt3= t3s[index//2]
                                      
    return(float(t1s[index])-t1)     


    t1s=np.array([1,4,2,5,6])
    error=float(t1s[index])

In [13]:
def solve_with_fallback(initial_guess,camName,a_pixel,b_pixel,c_pixel,AB_square = 4,AC_square = 8,BC_square = 4,filename='CamID.json'):
    with open(filename, 'r') as file:
        cameras = json.load(file)
    for camera in cameras:
        if (camera.get("name")) == camName:
            pixelVector=np.array(camera.get('pixelVector'))
            break
    try:
        # Use fsolve to find the root
        
        root, info, ier, msg = fsolve(t_to_tCheck, initial_guess, full_output=True,args=("EricsPad",a_pixel,b_pixel,c_pixel,AB_square,AC_square,BC_square,'CamID.json'))
        
        # Check if fsolve succeeded (ier == 1 means success)
        if ier != 1:
            raise FsolveError("fsolve failed to converge", root)
        
        # Verify if the solution meets our expectations (e.g., within tolerance)
        if not np.isclose(func(root), 0, atol=1e-2):
            raise FsolveError("Solution did not meet tolerance", root)
        
        print(f"Root found: {root[0]}")
        return [tt1,tt2,tt3]

        
    except ValueError as e:
        print(f"optimization unable to proceed ")
        A=a_pixel*pixelVector*tt1
        B=b_pixel*pixelVector*tt2
        C=c_pixel*pixelVector*tt3
        return  [A,B,C]

In [34]:
#let B be origin, A on +x, C on +y,S be camera
def set_camera_placing(camName,pic_Source,x_square = 4,xy_square = 8,y_square = 4,filename='CamID.json',initial_guess=40):
    with open('CamID.json', 'r') as file:
        cameras = json.load(file)
    for camera in cameras:
        if (camera.get("name")) == 'EricsPad':
            #pV=camera.get('pixelVector')
            pV=[1,1,1]
    RGB=RGB_location(pic_Source)
#    print(RGB)
    origin_pixel=[RGB[0][0]*pV[0],RGB[0][1]*pV[1],1.]
    x_pixel=[RGB[1][0]*pV[0],RGB[1][1]*pV[1],1]
    y_pixel=[RGB[2][0]*pV[0],RGB[2][1]*pV[1],1]
#    print(type(origin_pixel[0]))
#    print(type(x_pixel[0]))
    ABC=solve_with_fallback(initial_guess,camName,x_pixel,origin_pixel,y_pixel,x_square,xy_square,y_square,'CamID.json')
    BS=ABC[1][:]
    BA=np.array(ABC[0][:]-ABC[1][:], dtype=np.float64)
    BC=np.array(ABC[2][:]-ABC[1][:], dtype=np.float64)
    
    unitX=BA/np.sqrt(np.sum((BA)**2))
    unitY=BC/np.sqrt(np.sum((BC)**2))
    unitZ=np.cross(unitX,unitY)
    
    print(sum(unitX**2))
    print(sum(unitY**2))
    print(sum(unitZ**2))
    
    CamLoca=[float(np.dot(BS,unitX)),
             float(np.dot(BS,unitY)),
             float(np.dot(BS,unitZ))]
    print("location:",CamLoca)
    #dot product of (1,0,0) and unitX,unitY unitZ respectively
    CamOrien_X=[float(unitX[0]),
              float(unitY[0]),
              float(unitZ[0])]
    CamOrien_Y=[float(unitX[1]),
              float(unitY[1]),
              float(unitZ[1])]
    #in right hand system, z is on 
    CamOrien_Z=[-float(unitX[2]),
              -float(unitY[2]),
              -float(unitZ[2])]
    
    
    set_camera_location("EricsPad",CamLoca,'CamID.json')
    set_camera_orientation("EricsPad",CamOrien_X,CamOrien_Y,CamOrien_Z,'CamID.json')

In [36]:
set_camera_placing("EricsPad",'RGBcalibration.jpeg',AB_square ,AC_square ,BC_square ,'CamID.json',40)

Mean position of red pixels (R): (1191, 520)
Mean position of green pixels (G): (1190, 584)
Mean position of blue pixels (B): (1238, 522)
Error= [2.69585221] %
Error= [2.69585221] %
Error= [2.69585221] %
Error= [2.69585216] %
optimization unable to proceed 
1.0
1.0000000000000002
0.9715614453715764
location: [-35.53657575548585, 9.026509676087638, -23.512174593042428]
location for 'EricsPad' updated successfully!
orientation for 'EricsPad' updated successfully!
